<a href="https://colab.research.google.com/github/Demiarge/kaggle-platesv2/blob/main/kaggle_compi_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Verify gpu


In [1]:
import tensorflow as tf

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')


Num GPUs Available: 1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Kaggle input and unzipping data to use properly

In [2]:
# install Kagglee
!pip install kaggle

# Upload Kaggle API key
from google.colab import files
files.upload()

# Move API key to the app location
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the competition data
!kaggle competitions download -c platesv2

# Unzip the data
!unzip platesv2.zip -d /content/temp
!rm -rf platesv2.zip #dlt

# Unzip main data
!cd /content/temp/
!ls
!unzip /content/temp/plates.zip -d /content/platesv2
!rm -rf /content/temp/

# move data in data folder
!mv /content/platesv2/plates/ /content/data/
!rm -rf /content/platesv2/

Saving kaggle.json to kaggle.json
 85% 40.0M/47.0M [00:00<00:00, 112MB/s] 
100% 47.0M/47.0M [00:00<00:00, 112MB/s]
Archive:  platesv2.zip
  inflating: /content/temp/plates.zip  
  inflating: /content/temp/sample_submission.csv  
sample_data  temp
Archive:  /content/temp/plates.zip
   creating: /content/platesv2/plates/
  inflating: /content/platesv2/plates/.DS_Store  
   creating: /content/platesv2/__MACOSX/
   creating: /content/platesv2/__MACOSX/plates/
  inflating: /content/platesv2/__MACOSX/plates/._.DS_Store  
   creating: /content/platesv2/plates/test/
  inflating: /content/platesv2/plates/test/0071.jpg  
  inflating: /content/platesv2/plates/test/0717.jpg  
  inflating: /content/platesv2/plates/test/0703.jpg  
  inflating: /content/platesv2/plates/test/0065.jpg  
  inflating: /content/platesv2/plates/test/0059.jpg  
  inflating: /content/platesv2/plates/test/0515.jpg  
  inflating: /content/platesv2/plates/test/0273.jpg  
  inflating: /content/platesv2/plates/test/0267.jpg  
  i

In [ ]:
# Additional cell for use

!ls -la

# Import library

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import shutil


# data traning


In [4]:
# Paths
train_dir = '/content/data/train'
test_dir = '/content/data/test'

cleaned_dir = os.path.join(train_dir, 'cleaned')
dirty_dir = os.path.join(train_dir, 'dirty')


In [5]:
# data generator
# Define parameters
img_size = 224
batch_size = 16

# Train data generator
train_datagen = ImageDataGenerator(
    rescale=1.0/255,         # Normalize images to [0, 1]
    rotation_range=20,       # Random rotation
    width_shift_range=0.2,   # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,         # Random shearing
    zoom_range=0.2,          # Random zoom
    horizontal_flip=True,    # Flip images horizontally
    validation_split=0.2     # Reserve 20% for validation
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'  # Training subset
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'  # Validation subset
)

# Test data generator
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    test_dir,  # Dummy directory for test images
    target_size=(img_size, img_size),
    batch_size=1,
    class_mode=None,
    shuffle=False  # Keep the order for submission
)


Found 32 images belonging to 2 classes.
Found 8 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


# Model

In [6]:
# Load Pretrained ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Add custom layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification

model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [7]:
# Train the model
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=epochs
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 17s 6s/step - accuracy: 0.5625 - loss: 0.7138 - val_accuracy: 0.5000 - val_loss: 0.6992
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 0.7048
Epoch 3/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 192ms/step - accuracy: 0.4375 - loss: 0.7519 - val_accuracy: 0.5000 - val_loss: 0.7600
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 0.7662
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 220ms/step - accuracy: 0.4583 - loss: 0.8214 - val_accuracy: 0.5000 - val_loss: 0.7009
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 0.7138
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 192ms/step - accuracy: 0.5208 - loss: 0.7097 - val_accuracy: 0.5000 - val_loss: 0.7259
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 0.7318
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step - accuracy: 0.4375 - loss: 0.7661 - val_accuracy: 0.3750 - val_loss: 0.6992
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 

In [8]:
# tuning  optional
# Unfreeze some layers in the base model
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Retrain the model
fine_tune_epochs = 5
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=fine_tune_epochs
)


Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.4583 - loss: 0.7105 - val_accuracy: 0.3750 - val_loss: 0.6987
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 0.6973
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 239ms/step - accuracy: 0.6667 - loss: 0.6642 - val_accuracy: 0.3750 - val_loss: 0.7005
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2500 - val_loss: 0.6990
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 221ms/step - accuracy: 0.5625 - loss: 0.7030 - val_accuracy: 0.3750 - val_loss: 0.7031


# Evaluate and submission

In [10]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Path to the test folder
test_dir = '/content/data/test'
img_size = 224  # Same as training size

# List all image file names
test_images = sorted([os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith('.jpg')])

# Preprocess test images
test_data = []
for img_path in test_images:
    img = load_img(img_path, target_size=(img_size, img_size))  # Load and resize
    img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
    test_data.append(img_array)

test_data = np.array(test_data)


In [11]:
# Predict labels for the test data
predictions = model.predict(test_data)

# Convert probabilities to binary labels (0 = dirty, 1 = cleaned)
predicted_labels = (predictions > 0.5).astype(int).ravel()

# Print sample predictions
print(predicted_labels[:10])  # Show predictions for the first 10 test images


24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 194ms/step
[0 0 0 0 0 0 0 1 0 0]


In [12]:
import pandas as pd

# Prepare the submission dataframe
image_ids = [f"{i:04d}.jpg" for i in range(len(test_data))]
submission = pd.DataFrame({'id': image_ids, 'label': predicted_labels})

# Save to CSV
submission.to_csv('submission.csv', index=False)

# Download the file (if using Colab)
from google.colab import files
files.download('submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>